## 단순선형 회귀분석

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import statsmodels
import statsmodels.formula.api as smf

In [3]:
import seaborn as sns

iris = sns.load_dataset('iris')
iris.head(3)

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa


- smf.ols: 최소제곱법을 사용한 회귀분석

In [4]:
#
formula =  "sepal_length ~ sepal_width" #반응/종속변수 length, 설명/독립변수 width
data = iris
model = smf.ols(formula, data).fit() 
model

In [8]:
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:           sepal_length   R-squared:                       0.014
Model:                            OLS   Adj. R-squared:                  0.007
Method:                 Least Squares   F-statistic:                     2.074
Date:                Mon, 15 Jan 2024   Prob (F-statistic):              0.152
Time:                        04:05:42   Log-Likelihood:                -183.00
No. Observations:                 150   AIC:                             370.0
Df Residuals:                     148   BIC:                             376.0
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept       6.5262      0.479     13.628      0.000       5.580       7.473
sepal_width    -0.2234      0.155     -1.440      0.152      -0.530       0.083
==============================================================================
Omnibus:                        4.389   Durbin-Watson:                   0.952
Prob(Omnibus):                  0.111   Jarque-Bera (JB):                4.237
Skew:                           0.360   Prob(JB):                        0.120
Kurtosis:                       2.600   Cond. No.                         24.2
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

#### 해설
- Adj. R-squared: 수정된 결정계수 -> 수정결정계수가 1에 가까울수록 이상적인 모델이다. 해당 모델은 별로임.
- coef: 회귀계수
- Intercept의 coef: 상수항, 절편
- sepal_width의 coef: 기울기
- P>|t|: t 검정통계량에 대한 p 값
    + width의 p값은 0.152이므로 0.05보다 크다. 귀무가설을 채택한다.
    + 회귀분석에서 귀무가설은 <기울기가 0이다.>
    + 기울기가 0이다 -> 종속변수와 독립변수는 관계가 없다.

In [7]:
Intercept = model.params.Intercept
slope = model.params.sepal_width

print(Intercept, slope, sep = '\n')

6.526222550894483
-0.22336106112990034


## 예제

- 주어진 데이터를 이용하여 항암제를 투여 받은 환자들의 부작용 분포와 감기약을 투여 받은 환자들의 부작용 분포간 차이가 있는지 확인하시오.
- 감기약 투여 환자들의 부작용 발생 비율
    + 아픔 5% / 조금아픔 10% / 속쓰림 15% / 무증상 70%
- 항암제 투여 환자들의 관찰된 부작용

In [23]:
import pandas as pd

df = pd.read_csv("data/data_6_3_1.csv")

In [ ]:
print(df['항암제'].values.tolist()) #values: Numpy배열로 반환, tolist(): 파이썬 리스트로 변환

### 문제 1. 항암제 투여 환자들 중 '무증상' 비율을 0과 1사이로 구하시오. 

In [13]:
df['항암제'].value_counts()

항암제
무증상     11
조금아픔     4
아픔       3
속쓰림      2
Name: count, dtype: int64

In [14]:
df['항암제'].value_counts(normalize = True)

항암제
무증상     0.55
조금아픔    0.20
아픔      0.15
속쓰림     0.10
Name: proportion, dtype: float64

In [20]:
no_symptom = df['항암제'].value_counts(normalize = True).iloc[0]
print(no_symptom)

0.55


### 카이제곱 검정
- 두 범주형 변수에 대한 분석 방법
- 3가지 유형이 존재한다.
    + 적합도 검정
    + 동질성 검정
    + 독립성 검정
- 참고: https://www.jmp.com/ko_kr/statistics-knowledge-portal/chi-square-test/chi-square-goodness-of-fit-test.html

#### 적합도 검정
- 한 범주형 변수의 각 그룹별 비율이 특정 비율과 같은지 검정함.
- 표본 데이터가 전체 모집단을 대표하는지 평가
- [범주형 변수] vs [알려진 사실, 특정 기준]
- [Group 1, Group 2, ...] vs [p1 : p2 : ...]
- 관측 데이터가 특정 분포를 따르는 여부를 검정할 때도 사용

#### 동질성 검정
- 각 집단이 서로 유사한 성향을 갖는지 분석함
- 열 변수의 분포가 동질한지 검정
- 행 변수를 고정하고 각 범주별로 비율이 다른지 검정
- [범주형 변수 A: G1, G2, ...] vs [범주형 변수 B: G1, G2, ...]
- 예시) 남자 100명과 여자 300명을 부모집단으로 선호 음료 유형이 서로 동질적인가?

#### 독립성 검정
- 두 범주형 변수가 서로 독립적인지 확인
- 전제 조사 대상을 고정하고 조사 결과를 바탕으로 두 범주형 변수 간 연관관계가 유의한지 확인.
- [부 모집단: Population 1, Population 2, ...] vs [범주형 변수: G1, G2, ...]
- 대표본 가정이 필요함. (각 항의 기대 도수가 5보다 큼)
- 만약 대표본 가정을 만족하지 않으면 피션의 정확으로 검정 방법을 바꿔야 함

### 문제 2. 검정통계량
- 감기약의 부작용 비율과 항암제의 관찰 값이 통계적으로 유의미하게 차이가 있는지 카이제곱 검정을 이용하여 검정통계량을 구하시오. (적합도 검정)
- 가설검정
    + 귀무가설 : 감기약의 부작용과 항암제의 부작용은 동일하다.
    + 대립가설 : 감기약의 부작용과 항암제의 부작용은 다르다. 

In [33]:
total = len(df['항암제']) #항암제 표본의 크기 확인. 20.

ratios = [0.7, 0.1, 0.05, 0.15] #감기약의 부작용 비율, 순서는 df['항암제'].value_count 순서

val1 = df['항암제'].value_counts().tolist() #실제 개수
val2 = [ratio * total for ratio in ratios] #기대 개수: [ratios[0] * total, ratios[1] * total, ratios[2] * total, ratios[3] * total]

print(val1, val2)

[11, 4, 3, 2] [14.0, 2.0, 1.0, 3.0]


In [38]:
import scipy.stats as stats

statistic, pvalue = stats.chisquare(val1, val2) #카이제곱검정 함수
statistic #검정통계량

6.976190476190475

- 검정통계량을 통한 가설검정.
- 카이제곱 분포에서 이론적 값을 찾는다.
- 유의수준은 0.05, 자유도는 (범주 수 - 1)
- 위의 문제에서는 0.05에 자유도 3이다. (범주는 증상 심각도)

In [43]:
from scipy.stats import chi2

chi_squr_value = chi2.ppf(0.95, df = 3) #7.814727903251179

if statistic >= chi_squr_value:
    print('귀무가설 기각')
else:
    print('귀무가설 채택')    

귀무가설 채택


- 구분하자
- 카이제곱 검정: 검정통계량 >= 이론값 -> 기각
- t 검정, p 검정: 통계량/p값 >= 임계값/유의수준 -> 채택

### 문제 3. 검정통계량
- p value 구하기

In [45]:
print(pvalue)

0.07266054733847582


In [44]:
if pvalue >= 0.05:
    print('귀무가설 채택')
else:
    print('귀무가설 기각') 

귀무가설 채택
